### Sample program for grid search of hyper parameters  

#### Import libraries  

In [16]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.svm import SVR
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV, KFold, cross_val_score
from sklearn.pipeline import Pipeline
from joblib import dump

#### Parameters  

In [17]:
csv_in = '../ai-04/winequality-red_modified-utf8.txt'

#### Read CSV file  

In [18]:
df_all = pd.read_csv(csv_in, delimiter='\s+', skiprows=13, header=0)
print(df_all.shape)
print(df_all.info())
display(df_all.head())

(1599, 12)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed_acidity         1599 non-null   float64
 1   volatile_acidity      1599 non-null   float64
 2   citric_acid           1599 non-null   float64
 3   residual_sugar        1599 non-null   float64
 4   chlorides             1598 non-null   float64
 5   free_sulfur_dioxide   1599 non-null   float64
 6   total_sulfur_dioxide  1599 non-null   float64
 7   density               1597 non-null   float64
 8   pH                    1599 non-null   float64
 9   sulphates             1599 non-null   float64
 10  alcohol               1599 non-null   float64
 11  quality               1599 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 150.0 KB
None


,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,NaN,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,NaN,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,NaN,11.0,34.0,0.9978,3.51,0.56,9.4,5


#### Check rows with missing values  

In [19]:
display(df_all[df_all.isnull().any(axis=1)])

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol,quality
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,NaN,3.20,0.68,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,NaN,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,NaN,11.0,34.0,0.9978,3.51,0.56,9.4,5


#### Drop rows with missing values  

In [20]:
df = df_all.dropna().reset_index(drop=True)
print(df.shape)
display(df.head())

(1596, 12)


,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
2,7.4,0.66,0.00,1.8,0.075,13.0,40.0,0.9978,3.51,0.56,9.4,5
3,7.9,0.60,0.06,1.6,0.069,15.0,59.0,0.9964,3.30,0.46,9.4,5
4,7.3,0.65,0.00,1.2,0.065,15.0,21.0,0.9946,3.39,0.47,10.0,7


#### Get X and y  

In [21]:
X = df.iloc[:, :-1]  # explanatory variables
y = df['quality']  # objective variable
print('X:', X.shape)
display(X.head())
print('y:', y.shape)
print(y.head())

X: (1596, 11)


,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4
1,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8
2,7.4,0.66,0.00,1.8,0.075,13.0,40.0,0.9978,3.51,0.56,9.4
3,7.9,0.60,0.06,1.6,0.069,15.0,59.0,0.9964,3.30,0.46,9.4
4,7.3,0.65,0.00,1.2,0.065,15.0,21.0,0.9946,3.39,0.47,10.0


y: (1596,)
0    5
1    5
2    5
3    5
4    7
Name: quality, dtype: int64


#### Make pipeline and set parameters for grid search   

In [22]:
pipe = Pipeline([('scaler', MinMaxScaler(feature_range=(0, 1), copy=True)),
                 ('svr', SVR(kernel='rbf'))])

C_grid = [1e0, 1e1, 1e2, 1e3]  # np.logspace(0, 3, num=4)
g_grid = [1e-4, 1e-3, 1e-2, 1e-1]  # np.logspace(-4, -1, num=4)
e_grid = [1e-2, 1e-1, 1e0, 1e1, 1e2]  # np.logspace(-2, 2, num=5)

# Hyper paramet settings for grid search
param_grid = {
    'svr__C': C_grid,
    'svr__gamma' : g_grid,
    'svr__epsilon' : e_grid,
}

#### Preparation of objects for cross validation  

In [23]:
grid_cv = KFold(n_splits=4, shuffle=True, random_state=7)  # for grid search
gen_cv = KFold(n_splits=4, shuffle=True, random_state=11)  # for estimation of generalization performance

#### Define the grid search for hyper parameters  

In [24]:
gs = GridSearchCV(pipe, param_grid , cv=grid_cv, scoring='neg_mean_squared_error')

#### Estimation of generalization performance  

In [25]:
%%time
nested_score = cross_val_score(gs, X=X, y=y, cv=gen_cv,
                               scoring='neg_mean_squared_error')
print(nested_score)
print(np.sqrt(-nested_score.mean()))

[-0.4500097  -0.41172573 -0.41271116 -0.40072214]
0.647141547008095
Wall time: 46.5 s


#### Cross-validation to obtain the model with the best hyperparameter set (best estimator)  
- Note: gs_best is already fit to the whole data (X) in gs.fit(X,y)  

In [26]:
%%time
gs.fit(X, y)
gs_best = gs.best_estimator_

Wall time: 18.6 s


In [27]:
print(gs_best)

Pipeline(steps=[('scaler', MinMaxScaler()), ('svr', SVR(C=100.0, gamma=0.1))])


#### Save the best predictor  

In [28]:
tag = 'wine'
model_file = 'svr_best_{}.joblib'.format(tag)
dump(gs_best, model_file)

['svr_best_wine.joblib']

#### Demo for loading best SVR model from file and using it for predicton   

In [29]:
from joblib import load

svr_from_file = load(model_file)
y_pred_from_file = svr_from_file.predict(X)  # from file

#### Compare the results with original prediction results  

In [30]:
from sklearn.metrics import mean_squared_error

y_pred = gs_best.predict(X)  # original one
print(mean_squared_error(y_pred, y_pred_from_file))

0.0
